<a href="https://colab.research.google.com/github/sonali-debnath/musical-BCI/blob/main/FMRI_PROTOTYPICAL_MAML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PREPROCESSING THE DATASET**

In [ ]:
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import os

# Data:
data = {
    'onset': [0.01, 12, 24, 36, 48.01, 62, 74, 86, 98.01, 112, 122, 134, 144, 156, 166, 178, 190.02, 204, 216, 226, 236, 248.01, 262.01, 276, 288],
    'duration': [6] * 25,
    'genre': ['rocknroll', 'symphonic', 'rocknroll', 'metal', 'symphonic', 'country', 'country', 'ambient',
              'ambient', 'country', 'symphonic', 'symphonic', 'ambient', 'metal', 'metal', 'ambient',
              'rocknroll', 'country', 'rocknroll', 'ambient', 'symphonic', 'metal', 'country', 'metal',
              'rocknroll'],
    'trigger_ts': [1233.5005, 1245.4996, 1257.4997, 1269.5, 1281.5003, 1295.4996, 1307.4993, 1319.4994,
                   1331.499, 1345.4985, 1355.499, 1367.4987, 1377.4986, 1389.4985, 1399.4991, 1411.4989,
                   1423.4995, 1437.4986, 1449.4994, 1459.4995, 1469.4986, 1481.4986, 1495.4987, 1509.4989,
                   1521.4994]
}

df = pd.DataFrame(data)

# Specify the path to your 4D NIfTI file
nifti_file_path = 'D:/MRI Dataset/ds00113b/sub-01/func/sub-01_task-auditoryperception_run-01_bold.nii.gz'

# Load the NIfTI file
nifti_image = nib.load(nifti_file_path)

# Extract the data array
nifti_data = nifti_image.get_fdata()

# Create output folder if it doesn't exist
output_folder = 'output_images7'
os.makedirs(output_folder, exist_ok=True)

# Create a subfolder for "no music"
no_music_folder = os.path.join(output_folder, 'no_music')
os.makedirs(no_music_folder, exist_ok=True)

# Save images for "no music" condition (before music starts)
for i in range(25):
    plt.imshow(nifti_data[:, :, i, 0], cmap='gray')  # Assuming you want slices from the first time point
    plt.axis('off')

    # Save the image to the "no music" folder with a unique name
    image_filename = f"{no_music_folder}/no_music_slice_{i}.png"
    plt.savefig(image_filename, bbox_inches='tight', pad_inches=0.0)
    plt.clf()  # Clear the figure for the next iteration

# Iterate over the dataframe and save images for each genre
for index, row in df.iterrows():
    genre = row['genre']
    onset = row['onset']
    duration = row['duration']
    timestamp = row['trigger_ts']

    # Convert timestamp to slice number based on your logic
    slice_start = int((timestamp - df['trigger_ts'].min()) / (df['trigger_ts'].max() - df['trigger_ts'].min()) * (nifti_data.shape[2] - 26))

    # Ensure that slice_start is within bounds
    slice_start = max(0, min(slice_start, nifti_data.shape[2] - 26))

    # Create a subfolder for each genre
    genre_folder = os.path.join(output_folder, genre)
    os.makedirs(genre_folder, exist_ok=True)

    # Create a subplot for each slice
    for i in range(slice_start, slice_start + 25):
        plt.imshow(nifti_data[:, :, i, 0], cmap='gray')  # Assuming you want slices from the first time point
        plt.axis('off')

        # Save the image to the genre-specific folder with a unique name
        image_filename = f"{genre_folder}/{genre}_slice_{i}.png"
        plt.savefig(image_filename, bbox_inches='tight', pad_inches=0.0)
        plt.clf()  # Clear the figure for the next iteration

print(f"All images saved to {output_folder}.")


**SEGMENTATION**

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt

# Function to perform image segmentation
def perform_segmentation(image_path):
    # Read the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply a simple thresholding for segmentation
    _, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

    return binary_image

# Function to overlay segmented spots on the original image
def overlay_spots(original_image, segmented_image):
    # Convert original image to BGR for color overlay
    original_image_bgr = cv2.cvtColor(original_image, cv2.COLOR_GRAY2BGR)

    # Set the spots in the original image to red
    original_image_bgr[segmented_image > 0] = [255, 0, 0]  # Red color

    return original_image_bgr

# Path to the output folder containing genre-specific subfolders
output_folder = 'output_images7'

# Get a list of genre folders
genre_folders = [f for f in os.listdir(output_folder) if os.path.isdir(os.path.join(output_folder, f))]

# Iterate over each genre folder, perform segmentation, overlay spots, and display the results
for genre in genre_folders:
    # Get the path to one image in the genre folder
    genre_folder = os.path.join(output_folder, genre)
    image_files = [f for f in os.listdir(genre_folder) if f.endswith('.png')]

    if image_files:
        # Select the first image in the folder
        image_path = os.path.join(genre_folder, image_files[0])

        # Perform segmentation
        segmented_image = perform_segmentation(image_path)

        # Read the original image
        original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Overlay spots on the original image
        result_image = overlay_spots(original_image, segmented_image)

        # Display the original and overlaid images
        plt.figure(figsize=(12, 6))

        plt.subplot(1, 3, 1)
        plt.imshow(original_image, cmap='gray')
        plt.title(f"{genre} - Original")
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(segmented_image, cmap='gray')
        plt.title(f"{genre} - Segmented")
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(result_image)
        plt.title(f"{genre} - Active Area highlighted")
        plt.axis('off')

        plt.show()
    else:
        print(f"No images found in the folder for genre: {genre}")


**CNN**

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc
from sklearn.metrics import recall_score, f1_score
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import time

# Path to the folder containing genre-specific subfolders
input_folder = 'output_images7'

# Load images and labels
images = []
labels = []

# Get a list of genre folders
genre_folders = [f for f in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, f))]

for label, genre in enumerate(genre_folders):
    genre_folder = os.path.join(input_folder, genre)
    image_files = [f for f in os.listdir(genre_folder) if f.endswith('.png')]

    for image_file in image_files:
        image_path = os.path.join(genre_folder, image_file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (128, 128))  # Resize the images to a consistent size
        images.append(image)
        labels.append(label)

# Convert lists to NumPy arrays
images = np.array(images) / 255.0  # Normalize pixel values to the range [0, 1]
labels = np.array(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define the CNN model
model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(genre_folders), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape input for CNN (add the channel dimension)
X_train = X_train.reshape((-1, 128, 128, 1))
X_test = X_test.reshape((-1, 128, 128, 1))

# Train the model
start_time = time.time()
model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test))
train_time = time.time() - start_time

#Evaluate the model
start_time = time.time()
y_pred = model.predict(X_test)
inference_time = time.time() - start_time

# Convert predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred_labels)
conf_matrix = confusion_matrix(y_test, y_pred_labels)

# Calculate sensitivity, specificity, F1 score
if len(genre_folders) == 2:  # Binary classification
    tn, fp, fn, tp = conf_matrix.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    f1_score = 2 * tp / (2 * tp + fp + fn)
else:  # Multi-class classification
    sensitivity = recall_score(y_test, y_pred_labels, average='weighted')
    specificity = "Not applicable for multi-class"
    f1_score = f1_score(y_test, y_pred_labels, average='weighted')

# Calculate AUC
y_one_hot = keras.utils.to_categorical(y_test, num_classes=len(genre_folders))
roc_auc = roc_auc_score(y_one_hot, y_pred, multi_class='ovr')

# Print results
print(f"\nTest Accuracy: {accuracy}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")
print(f"F1 Score: {f1_score}")
print(f"AUC: {roc_auc}")
print(f"Training Time: {train_time} seconds")
print(f"Inference Time per Sample: {inference_time / len(y_test)} seconds")


**PROTYPICAL MAML**

In [ ]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.metrics import confusion_matrix

# Initialize variables for metrics
accuracy_list = []
sensitivity_list = []
specificity_list = []
f1_score_list = []
auc_list = []
train_time_list = []
inference_time_list = []
confusion_matrices = []

# Set your data directory
train_data_dir = 'output_images7'  # Replace with the path to your train data

# Set the number of classes
num_classes = 6

# Set other hyperparameters
learning_rate = 0.003
meta_step_size = 0.25
inner_batch_size = 25
eval_batch_size = 25
meta_iters = 1
inner_iters = 4
eval_interval = 1
img_height, img_width = 28, 28

# Data generator
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Prototypical Network architecture
def conv_bn(x):
    x = layers.Conv2D(filters=64, kernel_size=3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    return layers.ReLU()(x)

def build_prototypical_net(input_shape, num_classes):
    inputs = layers.Input(input_shape)
    x = conv_bn(inputs)
    x = conv_bn(x)
    x = conv_bn(x)
    x = conv_bn(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    return models.Model(inputs, outputs)

# Instantiate the model
model = build_prototypical_net((img_height, img_width, 3), num_classes)

# Compile the model
learning_rate = 0.001
optimizer = optimizers.SGD(learning_rate=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training loop
training = []

curr = time.time()

for meta_iter in range(meta_iters):
    frac_done = meta_iter / meta_iters
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    old_vars = model.get_weights()

    # Sample a few images for testing from the training set
    test_images, test_labels = next(train_generator)

    mini_dataset = iter(train_generator)

    for i in range(inner_iters):
        images, labels = next(mini_dataset)

        with tf.GradientTape() as tape:
            preds = model(images)
            loss = tf.keras.losses.categorical_crossentropy(labels, preds)

        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

    new_vars = model.get_weights()

    for var in range(len(new_vars)):
        new_vars[var] = old_vars[var] + ((new_vars[var] - old_vars[var]) * cur_meta_step_size)

    model.set_weights(new_vars)

    if meta_iter % eval_interval == 0:
        old_vars = model.get_weights()

        for i in range(inner_iters):
            with tf.GradientTape() as tape:
                preds = model(test_images)
                loss = tf.keras.losses.categorical_crossentropy(test_labels, preds)

            grads = tape.gradient(loss, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_weights))

        test_preds = model.predict(test_images)
        test_preds = tf.argmax(test_preds, axis=1).numpy()
        num_correct = (test_preds == tf.argmax(test_labels, axis=1).numpy()).sum()
        model.set_weights(old_vars)
        accuracy = num_correct / eval_batch_size
        accuracy_list.append(accuracy)

        # Calculate Confusion Matrix
        cm = confusion_matrix(np.argmax(test_labels, axis=1), test_preds)
        confusion_matrices.append(cm)

        # Calculate Sensitivity, Specififcity, and F1 Score
        tp = cm[1][1]
        fp = cm[0][1]
        fn = cm[1][0]
        sensitivity = tp / (tp + fp)
        specififcity = tp / (tp + fn)
        f1_score = 2 * (sensitivity * specififcity) / (sensitivity + specififcity)

        precision_list.append(sensitivity)
        recall_list.append(specififcity)
        f1_score_list.append(f1_score)

stop = time.time()
train_time_list.append(stop - curr)
print(f'Final Test accuracy: {accuracy * 100:.2f}%')
print(f'Final Time: {stop - curr}')

# Calculate average metrics
avg_accuracy = np.mean(accuracy_list)
avg_sensitivity = np.mean(sensitivity_list)
avg_specififcity = np.mean(specififcity_list)
avg_f1_score = np.mean(f1_score_list)

print(f"\nAverage Test Accuracy: {avg_accuracy}")
print(f"Average Sensitivity: {avg_sensitivity}")
print(f"Average Specififcity: {avg_specififcity}")
print(f"Average F1 Score: {avg_f1_score}")
print(f"Average Training Time: {np.mean(train_time_list)} seconds")
print(f"Average Inference Time per Sample: {np.mean(inference_time_list)} seconds")


In [ ]:
# Calculate and print correlation matrix
avg_confusion_matrix = np.mean(confusion_matrices, axis=0)
correlation_matrix = np.corrcoef(avg_confusion_matrix)

print("\nAverage Confusion Matrix:")
print(avg_confusion_matrix)

print("\nCorrelation Matrix:")
print(correlation_matrix)

# Visualize Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(avg_confusion_matrix.astype(int), annot=True, fmt='d', cmap='Blues', xticklabels=range(num_classes), yticklabels=range(num_classes))
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# Visualize Correlation Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', xticklabels=range(num_classes), yticklabels=range(num_classes))
plt.xlabel('Classes')
plt.ylabel('Classes')
plt.title('Correlation Matrix')
plt.show()

**BOLD SIGNALS**

In [ ]:
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import os

# Example data:
data = {
    'onset': [0.01, 12, 24, 36, 48.01, 62, 74, 86, 98.01, 112, 122, 134, 144, 156, 166, 178, 190.02, 204, 216, 226, 236, 248.01, 262.01, 276, 288],
    'duration': [6] * 25,
    'genre': ['rocknroll', 'symphonic', 'rocknroll', 'metal', 'symphonic', 'country', 'country', 'ambient',
              'ambient', 'country', 'symphonic', 'symphonic', 'ambient', 'metal', 'metal', 'ambient',
              'rocknroll', 'country', 'rocknroll', 'ambient', 'symphonic', 'metal', 'country', 'metal',
              'rocknroll'],
    'trigger_ts': [1233.5005, 1245.4996, 1257.4997, 1269.5, 1281.5003, 1295.4996, 1307.4993, 1319.4994,
                   1331.499, 1345.4985, 1355.499, 1367.4987, 1377.4986, 1389.4985, 1399.4991, 1411.4989,
                   1423.4995, 1437.4986, 1449.4994, 1459.4995, 1469.4986, 1481.4986, 1495.4987, 1509.4989,
                   1521.4994]
}

df = pd.DataFrame(data)

# Specify the path to your 4D NIfTI file
nifti_file_path = 'D:/MRI Dataset/ds00113b/sub-01/func/sub-01_task-auditoryperception_run-01_bold.nii.gz'

# Load the NIfTI file
nifti_image = nib.load(nifti_file_path)

# Extract the data array
nifti_data = nifti_image.get_fdata()

# Create a subfolder for BOLD signals
output_folder = 'bold_signals'
os.makedirs(output_folder, exist_ok=True)

# Iterate over the dataframe and extract BOLD signals for each genre
for index, row in df.iterrows():
    genre = row['genre']
    onset = row['onset']
    duration = row['duration']
    timestamp = row['trigger_ts']

    # Convert timestamp to volume number based on your logic
    volume_start = int((timestamp - df['trigger_ts'].min()) / (df['trigger_ts'].max() - df['trigger_ts'].min()) * (nifti_data.shape[3] - 25))

    # Ensure that volume_start is within bounds
    volume_start = max(0, min(volume_start, nifti_data.shape[3] - 25))

    # Extract BOLD signals for each volume in the duration
    bold_signals = nifti_data[:, :, :, volume_start:volume_start + 25].mean(axis=(0, 1, 2))

    # Plot and save the BOLD signal
    plt.plot(bold_signals, label=genre)
    plt.xlabel('Time (Volumes)')
    plt.ylabel('Mean BOLD Signal')
    plt.title(f'BOLD Signal for {genre}')
    plt.legend()

    # Save the plot to the output folder with a unique name
    plot_filename = f"{output_folder}/{genre}_bold_signal.png"
    plt.savefig(plot_filename, bbox_inches='tight', pad_inches=0.0)
    plt.clf()  # Clear the figure for the next iteration

print(f"All BOLD signals plotted and saved to {output_folder}.")


In [ ]:
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import os

# Data:
data = {
    'onset': [0.01, 12, 24, 36, 48.01, 62, 74, 86, 98.01, 112, 122, 134, 144, 156, 166, 178, 190.02, 204, 216, 226, 236, 248.01, 262.01, 276, 288],
    'duration': [6] * 25,
    'genre': ['Rocknroll', 'Symphonic', 'Rocknroll', 'Metal', 'Symphonic', 'Country', 'Country', 'Ambient',
          'Ambient', 'Country', 'Symphonic', 'Symphonic', 'Ambient', 'Metal', 'Metal', 'Ambient',
          'Rocknroll', 'Country', 'Rocknroll', 'Ambient', 'Symphonic', 'Metal', 'Country', 'Metal',
          'Rocknroll'],
    'trigger_ts': [1233.5005, 1245.4996, 1257.4997, 1269.5, 1281.5003, 1295.4996, 1307.4993, 1319.4994,
                   1331.499, 1345.4985, 1355.499, 1367.4987, 1377.4986, 1389.4985, 1399.4991, 1411.4989,
                   1423.4995, 1437.4986, 1449.4994, 1459.4995, 1469.4986, 1481.4986, 1495.4987, 1509.4989,
                   1521.4994]
}

df = pd.DataFrame(data)

# Specify the path to your 4D NIfTI file
nifti_file_path = 'D:/MRI Dataset/ds00113b/sub-01/func/sub-01_task-auditoryperception_run-01_bold.nii.gz'

# Load the NIfTI file
nifti_image = nib.load(nifti_file_path)

# Extract the data array
nifti_data = nifti_image.get_fdata()

# Create an empty array to accumulate BOLD signals for each genre
all_bold_signals = []

# Iterate over the dataframe and extract BOLD signals for each genre
for index, row in df.iterrows():
    genre = row['genre']
    onset = row['onset']
    duration = row['duration']
    timestamp = row['trigger_ts']

    # Convert timestamp to volume number based on your logic
    volume_start = int((timestamp - df['trigger_ts'].min()) / (df['trigger_ts'].max() - df['trigger_ts'].min()) * (nifti_data.shape[3] - 25))

    # Ensure that volume_start is within bounds
    volume_start = max(0, min(volume_start, nifti_data.shape[3] - 25))

    # Extract BOLD signals for each volume in the duration
    bold_signals = nifti_data[:, :, :, volume_start:volume_start + 25].mean(axis=(0, 1, 2))

    # Accumulate the BOLD signals for each genre
    all_bold_signals.append((genre, bold_signals))

# Plot all BOLD signals in a single plot with different markers and line styles
markers = ['o', 's', '^', 'D', '*', '2']
line_styles = ['-', '-', '-', '-', '-', ':']
for i, (genre, bold_signals) in enumerate(all_bold_signals):
    plt.plot(bold_signals, label=genre, marker=markers[i % len(markers)], linestyle=line_styles[i % len(line_styles)])

plt.xlabel('Time (Volumes) →')
plt.ylabel('Mean BOLD Signal →')
plt.title('BOLD Signals for Different Genres')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import os

# Data:
data = {
    'onset': [0.01, 12, 24, 36, 48.01, 62, 74, 86, 98.01, 112, 122, 134, 144, 156, 166, 178, 190.02, 204, 216, 226, 236, 248.01, 262.01, 276, 288],
    'duration': [6] * 25,
    'genre': ['Rocknroll', 'Symphonic', 'Rocknroll', 'Metal', 'Symphonic', 'Country', 'Country', 'Ambient',
          'Ambient', 'Country', 'Symphonic', 'Symphonic', 'Ambient', 'Metal', 'Metal', 'Ambient',
          'Rocknroll', 'Country', 'Rocknroll', 'Ambient', 'Symphonic', 'Metal', 'Country', 'Metal',
          'Rocknroll'],
    'trigger_ts': [1233.5005, 1245.4996, 1257.4997, 1269.5, 1281.5003, 1295.4996, 1307.4993, 1319.4994,
                   1331.499, 1345.4985, 1355.499, 1367.4987, 1377.4986, 1389.4985, 1399.4991, 1411.4989,
                   1423.4995, 1437.4986, 1449.4994, 1459.4995, 1469.4986, 1481.4986, 1495.4987, 1509.4989,
                   1521.4994]
}

df = pd.DataFrame(data)

# Specify the path to your 4D NIfTI file
nifti_file_path = 'D:/MRI Dataset/ds00113b/sub-01/func/sub-01_task-auditoryperception_run-01_bold.nii.gz'

# Load the NIfTI file
nifti_image = nib.load(nifti_file_path)

# Extract the data array
nifti_data = nifti_image.get_fdata()

# Create an empty array to accumulate BOLD signals for each genre
all_bold_signals = []

# Iterate over the dataframe and extract BOLD signals for each genre
for index, row in df.iterrows():
    genre = row['genre']
    onset = row['onset']
    duration = row['duration']
    timestamp = row['trigger_ts']

    # Convert timestamp to volume number based on your logic
    volume_start = int((timestamp - df['trigger_ts'].min()) / (df['trigger_ts'].max() - df['trigger_ts'].min()) * (nifti_data.shape[3] - 25))

    # Ensure that volume_start is within bounds
    volume_start = max(0, min(volume_start, nifti_data.shape[3] - 25))

    # Extract BOLD signals for each volume in the duration
    bold_signals = nifti_data[:, :, :, volume_start:volume_start + 25].std(axis=(0, 1, 2))

    # Accumulate the BOLD signals for each genre
    all_bold_signals.append((genre, bold_signals))

# Plot all BOLD signals in a single plot with different markers and line styles
markers = ['o', 's', '^', 'D', '*', '2']
line_styles = ['-', '-', '-', '-', '-', ':']
for i, (genre, bold_signals) in enumerate(all_bold_signals):
    plt.plot(bold_signals, label=genre, marker=markers[i % len(markers)], linestyle=line_styles[i % len(line_styles)])

plt.xlabel('Time (Volumes) →')
plt.ylabel('Standard Deviation BOLD Signal →')
plt.title('BOLD Signals Standard Deviation for Different Genres')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import os

# Data:
data = {
    'onset': [0.01, 12, 24, 36, 48.01, 62, 74, 86, 98.01, 112, 122, 134, 144, 156, 166, 178, 190.02, 204, 216, 226, 236, 248.01, 262.01, 276, 288],
    'duration': [6] * 25,
    'genre': ['Rocknroll', 'Symphonic', 'Rocknroll', 'Metal', 'Symphonic', 'Country', 'Country', 'Ambient',
          'Ambient', 'Country', 'Symphonic', 'Symphonic', 'Ambient', 'Metal', 'Metal', 'Ambient',
          'Rocknroll', 'Country', 'Rocknroll', 'Ambient', 'Symphonic', 'Metal', 'Country', 'Metal',
          'Rocknroll'],
    'trigger_ts': [1233.5005, 1245.4996, 1257.4997, 1269.5, 1281.5003, 1295.4996, 1307.4993, 1319.4994,
                   1331.499, 1345.4985, 1355.499, 1367.4987, 1377.4986, 1389.4985, 1399.4991, 1411.4989,
                   1423.4995, 1437.4986, 1449.4994, 1459.4995, 1469.4986, 1481.4986, 1495.4987, 1509.4989,
                   1521.4994]
}

df = pd.DataFrame(data)

# Specify the path to your 4D NIfTI file
nifti_file_path = 'D:/MRI Dataset/ds00113b/sub-01/func/sub-01_task-auditoryperception_run-01_bold.nii.gz'

# Load the NIfTI file
nifti_image = nib.load(nifti_file_path)

# Extract the data array
nifti_data = nifti_image.get_fdata()

# Create an empty array to accumulate BOLD signals for each genre
all_bold_signals = []

# Iterate over the dataframe and extract BOLD signals for each genre
for index, row in df.iterrows():
    genre = row['genre']
    onset = row['onset']
    duration = row['duration']
    timestamp = row['trigger_ts']

    # Convert timestamp to volume number based on your logic
    volume_start = int((timestamp - df['trigger_ts'].min()) / (df['trigger_ts'].max() - df['trigger_ts'].min()) * (nifti_data.shape[3] - 25))

    # Ensure that volume_start is within bounds
    volume_start = max(0, min(volume_start, nifti_data.shape[3] - 25))

    # Extract BOLD signals for each volume in the duration
    bold_signals = nifti_data[:, :, :, volume_start:volume_start + 25].max(axis=(0, 1, 2))

    # Accumulate the BOLD signals for each genre
    all_bold_signals.append((genre, bold_signals))

# Plot all BOLD signals in a single plot with different markers and line styles
markers = ['o', 's', '^', 'D', '*', '2']
line_styles = ['-', '-', '-', '-', '-', ':']
for i, (genre, bold_signals) in enumerate(all_bold_signals):
    plt.plot(bold_signals, label=genre, marker=markers[i % len(markers)], linestyle=line_styles[i % len(line_styles)])

plt.xlabel('Time (Volumes) →')
plt.ylabel('Maximum BOLD Signal →')
plt.title('BOLD Signals Maximum for Different Genres')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import os

# Data:
data = {
    'onset': [0.01, 12, 24, 36, 48.01, 62, 74, 86, 98.01, 112, 122, 134, 144, 156, 166, 178, 190.02, 204, 216, 226, 236, 248.01, 262.01, 276, 288],
    'duration': [6] * 25,
    'genre': ['Rocknroll', 'Symphonic', 'Rocknroll', 'Metal', 'Symphonic', 'Country', 'Country', 'Ambient',
          'Ambient', 'Country', 'Symphonic', 'Symphonic', 'Ambient', 'Metal', 'Metal', 'Ambient',
          'Rocknroll', 'Country', 'Rocknroll', 'Ambient', 'Symphonic', 'Metal', 'Country', 'Metal',
          'Rocknroll'],
    'trigger_ts': [1233.5005, 1245.4996, 1257.4997, 1269.5, 1281.5003, 1295.4996, 1307.4993, 1319.4994,
                   1331.499, 1345.4985, 1355.499, 1367.4987, 1377.4986, 1389.4985, 1399.4991, 1411.4989,
                   1423.4995, 1437.4986, 1449.4994, 1459.4995, 1469.4986, 1481.4986, 1495.4987, 1509.4989,
                   1521.4994]
}

df = pd.DataFrame(data)

# Specify the path to your 4D NIfTI file
nifti_file_path = 'D:/MRI Dataset/ds00113b/sub-01/func/sub-01_task-auditoryperception_run-01_bold.nii.gz'

# Load the NIfTI file
nifti_image = nib.load(nifti_file_path)

# Extract the data array
nifti_data = nifti_image.get_fdata()

# Create an empty array to accumulate BOLD signals for each genre
all_bold_signals = []

# Iterate over the dataframe and extract BOLD signals for each genre
for index, row in df.iterrows():
    genre = row['genre']
    onset = row['onset']
    duration = row['duration']
    timestamp = row['trigger_ts']

    # Convert timestamp to volume number based on your logic
    volume_start = int((timestamp - df['trigger_ts'].min()) / (df['trigger_ts'].max() - df['trigger_ts'].min()) * (nifti_data.shape[3] - 25))

    # Ensure that volume_start is within bounds
    volume_start = max(0, min(volume_start, nifti_data.shape[3] - 25))

    # Extract BOLD signals for each volume in the duration
    bold_signals = np.median(nifti_data[:, :, :, volume_start:volume_start + 25], axis=(0, 1, 2))

    # Accumulate the BOLD signals for each genre
    all_bold_signals.append((genre, bold_signals))

# Plot all BOLD signals in a single plot with different markers and line styles
markers = ['o', 's', '^', 'D', '*', '2']
line_styles = ['-', '-', '-', '-', '-', ':']
for i, (genre, bold_signals) in enumerate(all_bold_signals):
    plt.plot(bold_signals, label=genre, marker=markers[i % len(markers)], linestyle=line_styles[i % len(line_styles)])

plt.xlabel('Time (Volumes) →')
plt.ylabel('Median BOLD Signal →')
plt.title('BOLD Signals Median for Different Genres')
plt.legend()
plt.grid(True)
plt.show()


**CONNECTIVITY MATRIX**

In [ ]:
import pandas as pd
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

# Data:
data = {
    'onset': [0.01, 12, 24, 36, 48.01, 62, 74, 86, 98.01, 112, 122, 134, 144, 156, 166, 178, 190.02, 204, 216, 226, 236, 248.01, 262.01, 276, 288],
    'duration': [6] * 25,
    'genre': ['Rocknroll', 'Symphonic', 'Rocknroll', 'Metal', 'Symphonic', 'Country', 'Country', 'Ambient',
          'Ambient', 'Country', 'Symphonic', 'Symphonic', 'Ambient', 'Metal', 'Metal', 'Ambient',
          'Rocknroll', 'Country', 'Rocknroll', 'Ambient', 'Symphonic', 'Metal', 'Country', 'Metal',
          'Rocknroll'],
    'trigger_ts': [1233.5005, 1245.4996, 1257.4997, 1269.5, 1281.5003, 1295.4996, 1307.4993, 1319.4994,
                   1331.499, 1345.4985, 1355.499, 1367.4987, 1377.4986, 1389.4985, 1399.4991, 1411.4989,
                   1423.4995, 1437.4986, 1449.4994, 1459.4995, 1469.4986, 1481.4986, 1495.4987, 1509.4989,
                   1521.4994]
}

df = pd.DataFrame(data)

# Specify the path to your 4D NIfTI file
nifti_file_path = 'D:/MRI Dataset/ds00113b/sub-01/func/sub-01_task-auditoryperception_run-01_bold.nii.gz'

# Load the NIfTI file
nifti_image = nib.load(nifti_file_path)

# Extract the data array
nifti_data = nifti_image.get_fdata()

# Create a subfolder for displaying connectivity matrices
display_folder = 'display_matrices'
os.makedirs(display_folder, exist_ok=True)

# Iterate over the dataframe and extract BOLD signals for each genre
for index, row in df.iterrows():
    genre = row['genre']
    onset = row['onset']
    duration = row['duration']
    timestamp = row['trigger_ts']

    # Convert timestamp to volume number based on your logic
    volume_start = int((timestamp - df['trigger_ts'].min()) / (df['trigger_ts'].max() - df['trigger_ts'].min()) * (nifti_data.shape[3] - 25))

    # Ensure that volume_start is within bounds
    volume_start = max(0, min(volume_start, nifti_data.shape[3] - 25))

    # Extract BOLD signals for each volume in the duration
    bold_signals = nifti_data[:, :, :, volume_start:volume_start + 25]

    # Select a random voxel as the center of the ROI
    center_voxel = np.random.randint(0, bold_signals.shape[0]), np.random.randint(0, bold_signals.shape[1]), np.random.randint(0, bold_signals.shape[2])

    # Define the size of the ROI
    roi_size = 5

    # Extract the ROI
    roi = bold_signals[
        max(0, center_voxel[0] - roi_size): min(bold_signals.shape[0], center_voxel[0] + roi_size + 1),
        max(0, center_voxel[1] - roi_size): min(bold_signals.shape[1], center_voxel[1] + roi_size + 1),
        max(0, center_voxel[2] - roi_size): min(bold_signals.shape[2], center_voxel[2] + roi_size + 1),
        :
    ]

    # Reshape the signals for correlation calculation
    reshaped_signals = roi.reshape(-1, roi.shape[-1])

    # Compute the correlation matrix
    correlation_matrix = np.corrcoef(reshaped_signals)

    # Display a heatmap of the correlation matrix with color bar range 0 to 1
    plt.imshow(correlation_matrix, cmap='viridis', vmin=0, vmax=1)
    plt.colorbar()
    plt.title(f'{genre}')
    plt.xlabel('Source Voxels')
    plt.ylabel('Target Voxels')
    plt.show()
